In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import cv2
%matplotlib inline
print(tf.__version__)
from google.colab import drive
from sklearn.preprocessing import  MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
import pandas as pd

from tensorflow.keras.models import Model

# drive.mount("gdrive")

2.2.0-rc3
Drive already mounted at gdrive; to attempt to forcibly remount, call drive.mount("gdrive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
data = pd.read_csv("sample_data/california_housing_train.csv")

data.head()
x = data.iloc[:,2:-1]
y = data.iloc[:,-1]
x = x.values
y = y.values
# xtr,ytr,xte,yte = train_test_split(x,y,test_size=0.2,shuffle=True)

ms = MinMaxScaler()
xtrain = ms.fit_transform(x)
ytrain = ms.fit_transform(y.reshape(-1,1))

mshape = xtrain.shape[0]
batchsize = 128

dataset = tf.data.Dataset.from_tensor_slices((xtrain,ytrain))
dataset = dataset.shuffle(buffer_size=1024).batch(batchsize).repeat()



In [0]:
input_dim  = 8
hidden_size1 = 56
hidden_size2 = 56
output_dim = 1

inp_h1_w = tf.Variable(tf.random.normal(shape=(input_dim,hidden_size1)))
inp_h1_b = tf.Variable(tf.zeros((hidden_size1,)))
h1_h2_w  = tf.Variable(tf.random.normal(shape=(hidden_size1,hidden_size2)))
h1_h2_b  = tf.Variable(tf.zeros((hidden_size2,)))
h2_o_w   = tf.Variable(tf.random.normal(shape=(hidden_size2,output_dim)))
h2_o_b   = tf.Variable(tf.zeros((output_dim,)))

def regress(x):
  x  = tf.cast(x,tf.float32)
  comp1 = tf.matmul(x,inp_h1_w)+inp_h1_b
  comp1 = Activation("relu")(comp1)
  comp2 = tf.matmul(comp1,h1_h2_w)+h1_h2_b
  comp2 = Activation("relu")(comp2)
  comp3 = tf.matmul(comp2,h2_o_w)+h2_o_b
  comp3 = Activation("sigmoid")(comp3)
  return comp3


def compute_loss(labels, predictions):
  labels = tf.cast(labels,tf.float32)
  return tf.reduce_mean(tf.square(labels - predictions))

learning_rate = 0.00001

def train_on_batch(x,y):
  with tf.GradientTape(persistent=True) as t:
    func_out  = regress(x)
    loss  = compute_loss(y,func_out)
    if loss > 0.5:
      t.reset()
    dl_o,db_o   = t.gradient(loss,[h2_o_w,h2_o_b])
    dl_h2,db_h2 = t.gradient(loss,[h1_h2_w,h1_h2_b])
    dl_h1,db_h1 = t.gradient(loss,[inp_h1_w,inp_h1_b])
  h2_o_w.assign_sub(learning_rate * dl_o)
  h2_o_b.assign_sub(learning_rate * db_o)
  h1_h2_w.assign_sub(learning_rate*dl_h2)
  h1_h2_b.assign_sub(learning_rate*db_h2)
  inp_h1_w.assign_sub(learning_rate*dl_h1)
  inp_h1_b.assign_sub(learning_rate*db_h1)
  return tf.reduce_sum(loss)

for epoch in range(40):
  for step, (x, y) in enumerate(dataset):
    loss = train_on_batch(x, y)
    print("Epoch %d : loss = %.4f" %(step,loss))


In [0]:

class NN(Model):
  def __init__(self):
    super(NN,self).__init__()
    self.inp = InputLayer(input_shape=(6,))
    self.l1 = Dense(128,activation="relu")
    self.l2 = Dense(128,activation="relu")
    self.d = Dropout(0.1)
    self.b = BatchNormalization(axis=-1)
    self.out = Dense(1,activation="sigmoid")

  def call(self,inputs):
    x = self.inp(inputs)
    x = self.l1(x)
    x = self.d(x)
    x = self.b(x)
    x = self.l2(x)
    x = self.b(x)
    x = self.l2(x)
    o = self.out(x)
    return o



def NN():
  inp = Input(shape=(6,))
  d1 = Dense(64,activation="relu")(inp)
  b1= BatchNormalization()(d1)
  d2 = Dense(128,activation="relu")(b1)
  fp = Dropout(0.1)(d2)
  o  = Dense(1,activation="sigmoid")(fp)
  mod = Model(inp,o)
  return mod

  # dec.fit(dataset,epochs=30,steps_per_epoch=mshape//batchsize)

In [0]:
optim = Nadam(1e-3)
build = NN()

def train_batch(x,y):
  with tf.GradientTape() as tape:
    pred = build(x)
    loss = tf.keras.losses.mean_squared_error(y,pred)
    grad = tape.gradient(loss,build.trainable_variables)
    optim.apply_gradients(zip(grad,build.trainable_variables))
    return tf.reduce_mean(loss)

for j in range(1):
  for s,(x,y) in enumerate(dataset):
    outs = train_batch(x,y)
    print("epochs - %d | loss : %.4f"%(s,outs))